In [2]:
from langchain.document_loaders import PyPDFLoader
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

In [3]:
loader = PyPDFLoader("./Kurikulum Teknik Informatika - FILKOM 2020 v1.9.pdf")
pages = loader.load_and_split()
len(pages)

127

In [4]:
pages[100].page_content

'95  4 Preprocessing data 5 Data warehouse 6 Algoritme Asosiasi (Apriori) 7 Konsep Klasifikasi dan Evaluasi hasil klasifikasi 8 Konsep rule-based classifier dan algoritma Ripper 9 Konsep Metode Ensemble, algoritma Bootstrap dan AdaBoost 10 Konsep Clustering dan Algoritme DBSCAN 11 Algoritma Fuzzy C-Means 12 Konsep outlier dan anomali data, deteksi outlier dengan pendekatan statistik, proximity-based, dan cluster-based PUSTAKA UTAMA 1. Han, J., Kamber, M., Pei, J. “Data Mining Concept and Techniques”, 3rd ed., Morgan Kaufmann, 2012; 2. Tan, P., Steinbach, M., Kumar, V., “Introduction to Data Mining”, Pearson, 2006; PUSTAKA PENDUKUNG - PRASYARAT (Jika ada) Algoritma dan Struktur Data  (CIF62004)  NAMA MATA KULIAH Nama Pengolahan Citra Digital Kode MK CIF61048 Kredit (SKS) 3 Semester Ganjil DESKRIPSI MATA KULIAH Matakuliah ini didesain agar mahasiswa memahami, mendeskripsikan dan mampu mendemonstrasikan penerapan konsep dasar pengolahan citra digital. Konsep dasar yang dimaksud meliputi p

In [8]:
all_pages = ""

for page in pages:
    all_pages += page.page_content + "\n"

len(all_pages)

321268

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex = False,
)

texts = text_splitter.create_documents([all_pages])

In [12]:
len(texts)

434

In [17]:
len(texts[0].page_content), len(texts[1].page_content), len(texts[2].page_content)

(185, 861, 933)

In [31]:
import pandas as pd

def validate_jsonl(text):
    
    try:
        pd.read_json(text, lines=True)
        return True
    except:
        return False

validate_jsonl("""\
{"question": "Siapa anaknya pak Agus?", "answer": "Budi"}\n\
{"question": "Apa yang disukai Budi?", "answer": "bermain bola"}
halo""")

False

In [32]:
texts[0].page_content

'PEDOMAN PELAKSANAAN KURIKULUM PROGRAM STUDI TEKNIK INFORMATIKA JURUSAN TEKNIK INFORMATIKA FAKULTAS ILMU KOMPUTER       \n                   UNIVERSITAS BRAWIJAYA TAHUN AKADEMIK 2020/2021'

In [77]:
all_pages = ""

for page in pages[4:-1]: # lebih selektif dalam memilih halaman
    all_pages += page.page_content + "\n"

len(all_pages)

313020

In [78]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex = False,
)

texts = text_splitter.create_documents([all_pages])

In [79]:
template = """\
Buatlah pasangan pertanyaan dan jawaban dari konteks yang diberikan.\
Pertanyaan yang diberikan harus memiliki jawaban yang dapat ditemukan di dalam konteks.\
Buatlah pertanyaan sebanyak mungkin! \
Berikan pertanyaan yang jelas dan tidak ambigu! Jangan gunakan kata ganti seperti "ini", "itu", dll.\
Berikan penjelasan mata kuliah apa yang sedang ditanyakan! \
Berikan jawaban yang jelas! \
format dalam bentuk jsonl. Tidak perlu menambahkan teks lain selain JSONL.

# Contoh:
Konteks: "Mata kuliah Text Mining membahas tentang pengolahan teks. Total 3 SKS. CPMK M1: menganalisis karakteristik suatu proyek perangkat lunak"
    
# Hasil:
{{"question": "Apa yang dibahas mata kuliah Text Mining?", "answer": "pengolahan teks"}}
{{"question": "Berapa SKS mata kuliah Text Mining?", "answer": "3"}}
{{"question": "Apa yang dibahas pada CPMK M1? pada mata kuliah Text Mining?", "answer": "menganalisis karakteristik suatu proyek perangkat lunak"}}

Konteks: {konteks}

Hasil:
"""

prompt = PromptTemplate(template=template, input_variables=["konteks"])
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = OpenAI(
    model_name="gpt-3.5-turbo",
    callback_manager=callback_manager,
)

llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=False)
hasil = ""

with get_openai_callback() as cb:
    for i, text in enumerate(texts):
        retry = True
        n_retry = 1
        while retry:
            qa_pairs = llm_chain.run(konteks=text.page_content)
            if validate_jsonl(qa_pairs):
                print(f"split {i + 1} done!")
                sample = qa_pairs.split("\n")[0]
                print(f"sampel qa pairs: \n{sample}")
                hasil += "\n" + qa_pairs
                retry = False
            else:
                print(f"retrying split {i + 1} for {n_retry} times...")
                n_retry += 1    
                if n_retry == 3:
                    print("Failed to split text")
                    break

print(hasil)

/data/home/abiyyu/miniconda3/envs/biyu/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/data/home/abiyyu/miniconda3/envs/biyu/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


split 1 done!
sampel qa pairs: 
{"question": "Apa fungsi dari buku panduan Kurikulum Berbasis Standar Nasional Pendidikan Tinggi (SN-DIKTI)?", "answer": "sebagai panduan mahasiswa dalam menempuh jenjang pendidikan S1 Teknik Informatika Fakultas Ilmu Komputer (FILKOM) Universitas Brawijaya"}
split 2 done!
sampel qa pairs: 
{"question": "Apa nama organisasi yang menyusun buku panduan Kurikulum Prodi S1 Teknik Informatika?", "answer": "APTIKOM (Asosiasi Perguruan Tinggi Informatika dan Komputer)"}
split 3 done!
sampel qa pairs: 
{"question": "Apa tujuan dari prodi S1 Teknik Informatika?", "answer": "sebagai acuan pelaksanaan dan sumber informasi kepada seluruh sivitas akademika"}
split 4 done!
sampel qa pairs: 
{"question": "Siapa pengarah dari tim penyusun kurikulum?", "answer": "Wayan Firdaus Mahmudy, S.Si., M.T., Ph.D."}
split 5 done!
sampel qa pairs: 
{"question": "Kapan Prodi Teknik Informatika Universitas Brawijaya didirikan?", "answer": "29 Juni 2007"}
split 6 done!
sampel qa pairs

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


split 50 done!
sampel qa pairs: 
{"question": "Apa saja hal yang harus dilakukan oleh mahasiswa yang belum lulus mata kuliah pada kurikulum lama?", "answer": "mengambil mata kuliah yang baru di kurikulum 2020"}
split 51 done!
sampel qa pairs: 
{"question": "Apa saja perubahan yang terjadi pada MK wajib?", "answer": "berpraktikum"}
split 52 done!
sampel qa pairs: 
{"question": "Apa yang dimaksud dengan perubahan prasyarat mata kuliah?", "answer": "aturan peralihan yang mengubah prasyarat mata kuliah"}
split 53 done!
sampel qa pairs: 
{"question": "Apa prasyarat MK Basis Data pada semester Ganjil dan Genap 2020-2021?", "answer": "MK Pemrograman Dasar"}
split 54 done!
sampel qa pairs: 
{"question": "Apa yang menjadi fokus dari proyek kemanusiaan?", "answer": "Pemecahan masalah sosial, pemberian bantuan tenaga, menghasilkan dampak yang nyata"}
split 55 done!
sampel qa pairs: 
{"question": "Apa yang dilakukan setelah kegiatan ini?", "answer": "menjadi tenaga medis di tengah serangan wabah"}

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


split 175 done!
sampel qa pairs: 
{"question": "Apa yang dibahas pada CPMK M10?", "answer": "mengenai antar muka file system dan implementasinya"}
split 176 done!
sampel qa pairs: 
{"question": "Apa yang dibahas pada proses M5?", "answer": "kriteria dan algoritma penjadwalan"}
split 177 done!
sampel qa pairs: 
{"question": "Apa yang dibahas pada M10 pada mata kuliah File System?", "answer": "file system mounting, file sharing, dan protection"}
split 178 done!
sampel qa pairs: 
{"question": "Apa yang dibahas pada mata kuliah Kecerdasan Buatan?", "answer": "empat teknik dasar pemecahan masalah yang terdapat pada AI: Searching, Reasoning, Planning, dan Learning"}
split 179 done!
sampel qa pairs: 
{"question": "Apa yang harus dipertimbangkan dalam penggunaan teknik dan metode?", "answer": "permasalahan apa yang akan diselesaikan"}
split 180 done!
sampel qa pairs: 
{"question": "Apa yang dibahas pada sub-CPMK 1 pada mata kuliah kecerdasan buatan?", "answer": "Mampu memahami gambaran dari ke

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


split 252 done!
sampel qa pairs: 
{"question": "Apa yang dibahas pada Materi Pembelajaran 1?", "answer": "Ekonomi kreatif: a. Kewirausahaan digital b. Startup"}
split 253 done!
sampel qa pairs: 
{"question": "Apa yang dibahas pada Persona 4?", "answer": "Kebutuhan customer"}
split 254 done!
sampel qa pairs: 
{"question": "Apa nama mata kuliah yang sedang ditanyakan?", "answer": "Administrasi dan Manajemen Jaringan"}
split 255 done!
sampel qa pairs: 
{"question": "Apa yang bisa diterapkan di dalam autonomous system atau antar AS?", "answer": "protokol routing"}
split 256 done!
sampel qa pairs: 
{"question": "Apa yang dibahas pada L1?", "answer": "cara kerja routing dan cara kerja forwarding"}
split 257 done!
sampel qa pairs: 
{"question": "Apa yang dibahas pada Materi Pembelajaran 1?", "answer": "Routing dan forwarding"}
split 258 done!
sampel qa pairs: 
{"question": "Apa yang dibahas pada mata kuliah ini?", "answer": "konsep dasar dan implementasi berbagai mekanisme pengelolaan sebuah 

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


split 266 done!
sampel qa pairs: 
{"question": "Apa yang dibahas dalam mata kuliah ini?", "answer": "topik dan teknologi terkini dalam bidang jaringan komputer"}
split 267 done!
sampel qa pairs: 
{"question": "Apa yang sedang dibahas dalam perkuliahan?", "answer": "teknologi jaringan terkini"}
split 268 done!
sampel qa pairs: 
{"question": "Apa yang dibahas dalam perkuliahan M3?", "answer": "protokol-protokol komunikasi terkait dengan teknologi jaringan terkini"}
split 269 done!
sampel qa pairs: 
{"question": "Apa yang sedang dibahas dalam perkuliahan mata kuliah Jaringan Nirkabel?", "answer": "teknologi jaringan terkini"}
split 270 done!
sampel qa pairs: 
{"question": "Berapa jumlah SKS mata kuliah Jaringan Nirkabel?", "answer": "3"}
split 271 done!
sampel qa pairs: 
{"question": "Apa yang menjadi subjek pembahasan mata kuliah ini?", "answer": "jaringan selular"}
split 272 done!
sampel qa pairs: 
{"question": "Apa yang akan dipelajari pada mata kuliah ini?", "answer": "prinsip jaringa

In [81]:
cb

Tokens Used: 331000
	Prompt Tokens: 237771
	Completion Tokens: 93229
Successful Requests: 424
Total Cost (USD): $0.5431144999999997

In [ ]:
with open("qa-kurikulum-v2.jsonl", "w") as f:
    f.write(hasil)